# Contextual layers

Get contextual layers from source and save them as `MBTiles` or `COGs`

### Python libraries

In [1]:
import os
import shutil
import datetime
import subprocess
import urllib.request
import zipfile
from tqdm import tqdm

import pandas as pd
import geopandas as gpd

### Utils

**create_mbtiles**

In [2]:
def create_mbtiles(source_path, dest_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel"):
    """
    Use tippecanoe to create a MBTILE at dest_path from source_path.
    layer_name is used for the name of the layer in the MBTILE.
    Regex file path (/*.geojson) is supported for source_path.
    """
    cmd = f"tippecanoe -o {dest_path} -l {layer_name} {opts} {source_path}"
    print(f"Processing: {cmd}")
    r = subprocess.call(cmd, shell=True)
    if r == 0:
        print("Task created")
    else:
        print("Task failed")
    print("Finished processing")

**create_COG**

In [10]:
def create_COG(source_path, dest_path, file_name, description):
    """
    Use gdal_translate to create a COG at dest_path from source_path.
    """
    # Include internal overviews
    #cmd_internal_overviews = f"gdaladdo -r average {source_path} 2 4 8 16"
    #cmd_create_cog = f"gdal_translate -co TILED=YES -co COPY_SRC_OVERVIEWS=YES -co COMPRESS=LZW {source_path} {dest_path}"
    cmd_create_cog =  f"gdal_translate -of COG -co TILING_SCHEME=GoogleMapsCompatible -co COMPRESS=LZW -co ZOOM_LEVEL_STRATEGY='lower' {source_path} {dest_path}"
    #cmd_create_cog =  f"gdalwarp -t_srs EPSG:4326 -co TILED=YES -co COMPRESS=LZW -co COPY_SRC_OVERVIEWS=YES {source_path} {dest_path}"
    cmd_metadata = f"gdal_edit.py -mo \"TIFFTAG_DOCUMENTNAME={file_name}\" -mo \"TIFFTAG_IMAGEDESCRIPTION={description}\" {dest_path}"
    for cmd in [cmd_create_cog]:
        print(f"Processing: {cmd}")
        r = subprocess.call(cmd, shell=True)
        if r == 0:
            print("Task created")
        else:
            print("Task failed")

    print("Finished processing")

## Read data from different sources
### World Database on Protected Areas ([source](https://www.protectedplanet.net/en))

In [4]:
data_folder = 'data/contextual_layers/'
if not os.path.exists(os.path.join(data_folder, 'raw')):
    os.makedirs(os.path.join(data_folder, 'raw'))
    
now = datetime.datetime.now()
month_name = now.strftime("%b")
year = now.year

date = f"{month_name}{year}"

file_name = os.path.join(data_folder, 'raw', f"WDPA_WDOECM_{date}_Public_all_shp.zip")
url = f"https://d1gam3xoknrgr2.cloudfront.net/current/WDPA_WDOECM_{date}_Public_all_shp.zip"

# Download the file with progress bar
with tqdm(unit='B', unit_scale=True, unit_divisor=1024, miniters=1, desc=url.split('/')[-1]) as t:
    urllib.request.urlretrieve(url, filename=file_name, reporthook=lambda x, y, z: t.update(y))

# Extract the contents of the zip file
with zipfile.ZipFile(file_name, 'r') as zip_ref:
    zip_ref.extractall(os.path.join(data_folder, 'raw'))
    
# Read and process shapefile
file_name = os.path.join(data_folder, 'raw', f'WDPA_WDOECM_{date}_Public_all_shp_1.zip')
with zipfile.ZipFile(file_name, 'r') as zip_ref:
    zip_ref.extractall(os.path.join(data_folder, 'raw'))
    shp_file = [f for f in zip_ref.namelist() if f.endswith('.shp')][0]
    with tqdm(total=1, desc=f'Reading and simplifying shapefile') as pbar:
        df = gpd.read_file(os.path.join(data_folder, 'raw', shp_file), callback=lambda x: pbar.update(1))
        df = df.to_crs('epsg:4326')
        df['geometry'] = df['geometry'].simplify(tolerance=0.001, 
                                                preserve_topology=True)
        df = df[['WDPAID', 'NAME', 'IUCN_CAT', 'STATUS', 'DESIG_TYPE',
                    'GOV_TYPE', 'OWN_TYPE', 'MANG_AUTH', 'geometry']]

# Use shutil.rmtree to remove the data_folder and all its contents
shutil.rmtree(os.path.join(data_folder, 'raw'))

# Save the final GeoDataFrame as a GeoJSON file
output_file = os.path.join(data_folder, 'protected_areas.json')
df.to_file(output_file, driver='GeoJSON')

WDPA_WDOECM_May2023_Public_all_shp.zip: 99.4MB [00:07, 13.6MB/s]


KeyboardInterrupt: 

### River basins 
- Major hydrological basins of the world ([source](https://data.apps.fao.org/catalog//iso/7707086d-af3c-41cc-8aa5-323d8609b2d1))
- Hydrological basins of the world ([source](https://data.apps.fao.org/catalog/iso/f2615a41-6383-4aa4-aa21-743330eb03ae))

In [16]:
data_folder = 'data/contextual_layers/'
if not os.path.exists(os.path.join(data_folder, 'raw')):
    os.makedirs(os.path.join(data_folder, 'raw'))
    
file_name = os.path.join(data_folder, 'raw', "hydro_basins_world.zip")
url = "https://storage.googleapis.com/810c63d8-3fde-4ecd-9882-14d62e3058be/static/downloads/HydroBasins/hydro_basins_world.zip"

# Download the file with progress bar
with tqdm(unit='B', unit_scale=True, unit_divisor=1024, miniters=1, desc=url.split('/')[-1]) as t:
    urllib.request.urlretrieve(url, filename=file_name, reporthook=lambda x, y, z: t.update(y))

# Extract the contents of the zip file
with zipfile.ZipFile(file_name, 'r') as zip_ref:
    zip_ref.extractall(os.path.join(data_folder, 'raw'))
    
# Read shapefile
shp_file = "SUB_BAS.shp"
with tqdm(total=1, desc=f'Reading shapefile') as pbar:
    df = gpd.read_file(os.path.join(data_folder, 'raw', shp_file), callback=lambda x: pbar.update(1))
    
# Use shutil.rmtree to remove the data_folder and all its contents
shutil.rmtree(os.path.join(data_folder, 'raw'))

# Save the final GeoDataFrame as a GeoJSON file
output_file = os.path.join(data_folder, 'hydrological_basins.json')
df.to_file(output_file, driver='GeoJSON')

hydro_basins_world.zip: 54.6MB [00:06, 8.21MB/s]
Reading shapefile:   0%|          | 0/1 [00:08<?, ?it/s]


### Irrecoverable carbon in Earth's ecosystems ([source](https://zenodo.org/record/4091029#.ZEI9ctKZOUl))

In [11]:
data_folder = 'data/contextual_layers/'
if not os.path.exists(os.path.join(data_folder, 'raw')):
    os.makedirs(os.path.join(data_folder, 'raw'))
    
file_name = os.path.join(data_folder, 'raw', "Irrecoverable_Carbon_2018.zip")
url = "https://zenodo.org/record/4091029/files/Irrecoverable_Carbon_2018.zip?download=1"

# Download the file with progress bar
with tqdm(unit='B', unit_scale=True, unit_divisor=1024, miniters=1, desc=url.split('/')[-1]) as t:
    urllib.request.urlretrieve(url, filename=file_name, reporthook=lambda x, y, z: t.update(y))
    
# Extract the contents of the zip file
with zipfile.ZipFile(file_name, 'r') as zip_ref:
    zip_ref.extractall(os.path.join(data_folder, 'raw'))
    
# Move the file to the destination directory
tif_file = "Irrecoverable_Carbon_2018/Irrecoverable_C_Total_2018.tif"
shutil.move(os.path.join(data_folder, 'raw', tif_file), data_folder)

# Use shutil.rmtree to remove the data_folder and all its contents
shutil.rmtree(os.path.join(data_folder, 'raw'))

Irrecoverable_Carbon_2018.zip?download=1: 1.88GB [17:10, 1.96MB/s]


### Global Gridded Relative Deprivation Index ([source](https://sedac.ciesin.columbia.edu/data/set/povmap-grdi-v1/data-download))

In [8]:
data_folder = 'data/contextual_layers/'
if not os.path.exists(os.path.join(data_folder, 'raw')):
    os.makedirs(os.path.join(data_folder, 'raw'))
    
file_name = os.path.join(data_folder, 'raw', "povmap-grdi-v1-grdiv1-geotiff.zip")
url = "https://sedac.ciesin.columbia.edu/downloads/data/povmap/povmap-grdi-v1/povmap-grdi-v1-grdiv1-geotiff.zip"

# Download the file with progress bar
with tqdm(unit='B', unit_scale=True, unit_divisor=1024, miniters=1, desc=url.split('/')[-1]) as t:
    urllib.request.urlretrieve(url, filename=file_name, reporthook=lambda x, y, z: t.update(y))
    
# Extract the contents of the zip file
with zipfile.ZipFile(file_name, 'r') as zip_ref:
    zip_ref.extractall(os.path.join(data_folder, 'raw'))
    
# Move the file to the destination directory
tif_file = "povmap-grdi-v1-grdiv1-geotiff/povmap-grdi-v1.tif"
shutil.move(os.path.join(data_folder, 'raw', tif_file), data_folder)

# Use shutil.rmtree to remove the data_folder and all its contents
shutil.rmtree(os.path.join(data_folder, 'raw'))

povmap-grdi-v1-grdiv1-geotiff.zip: 24.0kB [00:01, 23.3kB/s]


BadZipFile: File is not a zip file

## Create `MBTiles`

In [9]:
layers = {'Hydrological basins': 'hydrological_basins.json',
          'Protected areas': 'protected_area_simp.json'}

data_folder = 'data/contextual_layers'
if not os.path.exists(os.path.join(data_folder, 'mbtiles')):
    os.makedirs(os.path.join(data_folder, 'mbtiles'))

for layer_name, file in layers.items():
    print(layer_name)
    source_path = os.path.join(data_folder, file)
    dest_path = os.path.join(data_folder, 'mbtiles', file).split('.')[0]+".mbtiles"
    create_mbtiles(source_path, dest_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel")

Protected areas
Processing: tippecanoe -o data/contextual_layers/mbtiles/protected_areas.mbtiles -l Protected areas -zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel data/contextual_layers/protected_areas.json


areas: No such file or directory
data/contextual_layers/protected_areas.json:16975: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","name":"protected_areas","features":[]}
data/contextual_layers/protected_areas.json:3366: Found ] at top level
217424 features, 100418953 bytes of geometry, 11678921 bytes of separate metadata, 15612546 bytes of string pool
Choosing a maxzoom of -z4 for features about 18791 feet (5728 meters) apart
Choosing a maxzoom of -z8 for resolution of about 1567 feet (477 meters) within features
tile 0/0/0 size is 1077078 with detail 12, >500000    
Going to try keeping the sparsest 41.78% of the features to make it fit
tile 0/0/0 size is 963302 with detail 12, >500000    
Going to try keeping the sparsest 19.52% of the features to make it fit
tile 0/0/0 size is 834220 with detail 12, >500000    
Going to try keeping the sparsest 10.53% of the features to make it fit
tile 0/0/0 size is 671448 with detail 12, >500000    
Goi

Task created
Finished processing


## Create `COGs`

In [11]:
layers = {'Irrecoverable carbon': 'Irrecoverable_C_Total_2018.tif',
          'Relative deprivation index': 'povmap-grdi-v1.tif'}

rename_file = {'Irrecoverable carbon': 'irrecoverable_carbon.tif',
               'Relative deprivation index': 'relative_deprivation_index.tif'}

data_folder = 'data/contextual_layers'
if not os.path.exists(os.path.join(data_folder, 'cogs')):
    os.makedirs(os.path.join(data_folder, 'cogs'))

for layer_name, input_file in layers.items():
    print(layer_name)
    output_file = rename_file[layer_name]
    source_path = os.path.join(data_folder, input_file)
    dest_path = os.path.join(data_folder, 'cogs', output_file)
    create_COG(source_path, dest_path, output_file, layer_name)

Irrecoverable carbon
Processing: gdal_translate -of COG -co TILING_SCHEME=GoogleMapsCompatible -co COMPRESS=LZW -co ZOOM_LEVEL_STRATEGY='lower' data/contextual_layers/Irrecoverable_C_Total_2018.tif data/contextual_layers/cogs/irrecoverable_carbon.tif
Input file size is 133584, 53434
0...10...20...30...40...50...60...70...80...90...100 - done.
Task created
Finished processing
Relative deprivation index
Processing: gdal_translate -of COG -co TILING_SCHEME=GoogleMapsCompatible -co COMPRESS=LZW -co ZOOM_LEVEL_STRATEGY='lower' data/contextual_layers/povmap-grdi-v1.tif data/contextual_layers/cogs/relative_deprivation_index.tif
Input file size is 43178, 16580
0...10...20...30...40...50...60...70...80...90...100 - done.
Task created
Finished processing
